In [25]:
# NAO USAR A TABELA data_product, E SIM A store_product!!

In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('inputs/data_product_202208091047.csv', usecols = ['id', 'barcode', 'name'])

In [28]:
df.shape

(69111, 3)

In [29]:
df['name'].nunique()


48879

In [30]:
df.drop_duplicates(subset = 'name', inplace = True)

In [31]:
df.shape

(48879, 3)

In [32]:
df['name'] = df['name'].astype(str)

In [33]:
df['name'] = df['name'].str.upper()

In [34]:
df['cleaned'] = df['name']

In [35]:
endswith_ = {
    'G':    -len('G'),
    'L':    -len('L'),
    'UN':   -len('UN'),
    'ML':   -len('ML'),
    'CPS':  -len('CPS'),
}

In [36]:
def remove_endswith(name, numbers):
    return lambda words: words.replace(''.join([x for x in words.split(' ') if x.endswith(name) and x[:numbers].isdigit()]), '')
    

In [37]:
for k, v in endswith_.items():
    df['cleaned'] = df['cleaned'].apply(remove_endswith(f'{k}', v))

In [38]:
df['cleaned'] = df['cleaned'].str.strip()

In [39]:
df['num_words'] = df['cleaned'].apply(lambda x: len(str(x).split(' ')))

In [40]:
df[df['id'].isin([57, 58, 60, 94, 76])]

,id,barcode,name,cleaned,num_words
56,57,7891721017377,CR GEL UNIVERSAL FPS20 45G,CR GEL UNIVERSAL FPS20,4
57,58,7891800348361,CURAT CREMER BARBIE 25UN,CURAT CREMER BARBIE,3
59,60,7896040704120,AMACIANTE VIDA MACIA 1L,AMACIANTE VIDA MACIA,3
75,76,7891142142979,EPIDRAT SENSI SH 120ML,EPIDRAT SENSI SH,3
93,94,7897700002013,ACEROLA E ROSA SILVESTRE 80CPS,ACEROLA E ROSA SILVESTRE,4


---

In [41]:
# df['cleaned'] = df['cleaned'].apply(lambda words: 
#     words.replace(''.join([x for x in words.split(' ') if 'C/' in x]), '')
# )

In [42]:
# df['cleaned'] = df['cleaned'].apply(lambda words: 
#     words.replace(''.join([x for x in words.split(' ') if ['R', 'REF'] in x]), '')
# )

In [43]:
# df['cleaned'] = df['cleaned'].apply(lambda words: 
#     words.replace(''.join([x for x in words.split(' ') if x.isdigit()]), '')
# )

In [44]:
# REMOVER:
# endswith / followed by
# unused_words
# startswith / followed by)
# endswith / followed by

In [45]:
# dict_ = {
#     'SAB': 'SABONETE',
#     'DES': 'DESODORANTE',
#     'ESC': 'ESCOVA',
#     'LOC': 'LOCAO',
#     'TRAD': 'TRADICIONAL',
#     'CURAT': 'CURATIVO',
# }